In [ ]:
import matplotlib.patches as mpatches
from matplotlib import rc
import numpy as np
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
params = {
   'axes.labelsize': 12,
   'font.size': 12,
   'legend.fontsize': 11,
   'xtick.labelsize': 12,
   'ytick.labelsize': 12,
   'text.usetex': False,
   'figure.figsize': [7, 2], # instead of 4.5, 4.5
   }
plt.rcParams.update(params)

## helper function

In [ ]:
def autolabel(rects, ses):
    # attach some text labels
    for rect, se in zip(rects, ses):
        height = rect.get_height()
        plt.axes().text(rect.get_x()+rect.get_width()/2., height+1.03*se, '{:2.3f}'.format(float(height)).lstrip('0'),
                ha='center', va='bottom',fontsize=10)

## Fixed Parameter

In [ ]:
# experiment
exp = 'loo_ds'
exp_label = 'Transfer Learning to an Unseen Prm. Dataset'

# dictionaries
cl_dict = {0:1/450,1:1/297,2:1/297,3:1/1973} # chance level of each dataset
feat_dict = {'multi_srm':[75,75,100],'all_srm':[75,75,50],'indv_srm':[75,75,50],'all_ica':[50,25,25],'indv_ica':[50,25,25],\
'all_gica':[100,50,25],'indv_gica':[100,50,25],'all_dict':[25,25,25],'indv_dict':[25,25,25]}
roi_dict = {'dmn':0,'pt':1,'eac':2}
# change multi_srm name!
md_dict = {'avg':'MNI','multi_srm':'MDSRM','ica':'ICA','gica':'Group-ICA','dict':'Dict','srm':'SRM'}
ds_dict = {0:'GreenEyes',1:'Milky',2:'Vodka',3:'Sherlock'}
lg_lb  = ['$1$ sec. dataset','$2$ sec. datasets'] # legend label

# paths
input_file = '../../output/accu_bar/{}/{}/{}_loo{}_other{}.npz' #exp,roi,model,loo,other
output_path = '../../output/figures/{}/' #exp
if not os.path.exists(output_path.format(exp)):
    os.makedirs(output_path.format(exp))
output_file = output_path+'{}_{}' #model,roi

## Plotting Parameter

In [ ]:
roi_all = ['dmn','pt','eac']
model = 'multi_srm'
max_accu = [0.23,0.25,0.25]
loo_ds = [0,3,1]
other_ds = [[1,3],[1,0],[0,3]] # length must be 2

## Aggregate Accuracy

In [ ]:
# Accuracies
all_mean = [] # length # of roi
all_se = [] # length # of roi
# aggregate each roi
for roi in roi_all:
    roi_mean = []
    roi_se = []
    for loo,other in zip(loo_ds,other_ds):
        ws=np.load(input_file.format(exp,roi,model,loo,other).replace(' ',''))
        roi_mean.extend(list(ws['mean']))
        roi_se.extend(list(ws['se']))
    all_mean.append(roi_mean)
    all_se.append(roi_se)
# Process xtick labels
lb1 = '$d_{}$\n$d_{}$'
lb2 = '$d_{}$\n$d_{},d_{}$'
xtick_name = []
# Add primary,secondary label
xtick_name.append('Prm.\nSec.')
for loo,other in zip(loo_ds,other_ds):
    xtick_name.append(lb1.format(loo,other[0]).replace('3','2'))
    xtick_name.append(lb2.format(loo,other[0],other[1]).replace('3','2'))

## Plot all roi

In [ ]:
width=1
nmodel = 2 # train with one/two datasets
ndata = len(loo_ds)
group_width = 1.3*width*nmodel
center_all = np.linspace(0,group_width*(ndata-1),ndata)
# set colors
color_all = ['dodgerblue','red']

for m,roi in enumerate(roi_all):
    xtick_idx = []
    xtick_idx.append(center_all[0]-0.5*width-0.1)
    plt.figure()
    # configure bars
    for i,center in enumerate(center_all):
        mean = list(all_mean[m][i*nmodel:(i+1)*nmodel])
        se   = list(all_se[m][i*nmodel:(i+1)*nmodel])    
        idx = np.arange(center,center+nmodel*width,width)
        error_config = {'ecolor': '0','capsize':3}   
        rects = plt.bar(idx, mean, yerr=se, align='center', error_kw=error_config, width = width-0.1)
        # set colors
        for rect_i in range(len(rects)):
            rects[rect_i].set_color(color_all[rect_i])
        autolabel(rects, se)
        xtick_idx.extend(idx)                             
    plt.xticks(xtick_idx,xtick_name)
    
    plt.ylabel('Accuracy')
    left_lim = center_all[0]-0.5*width-0.1
    right_lim = center_all[-1]+(nmodel-0.5)*width+0.15
    plt.xlim([left_lim,right_lim])
    plt.ylim([0,max_accu[m]])

    # plot chance accuracy of each dataset
    for d,center in enumerate(center_all):
        cl = cl_dict[loo_ds[d]]
        line = plt.plot([center-0.5*width, center+(nmodel-0.5)*width], [cl, cl], 'k-.', linewidth=2)
       
    # Add texts
    plt.text((right_lim+left_lim)/2, 1.23*max_accu[m], exp_label+' (ROI: '+roi.upper()+')',fontsize=13,horizontalalignment='center', verticalalignment='bottom')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#     plt.text((right_lim+left_lim)/2, 0.94*max_accu, 'k = '+str(feat_dict[model][roi_dict[roi]]),bbox=props,fontsize=12,horizontalalignment='center', verticalalignment='top')
    plt.text(left_lim+0.34*(right_lim-left_lim), 0.75*max_accu[m], 'k = '+str(feat_dict[model][roi_dict[roi]]),bbox=props,fontsize=11,horizontalalignment='center', verticalalignment='top')
    props2 = dict(boxstyle='square', facecolor='ivory',alpha=0.5)
#     ds_text = '$d_0$:GreenEyes\n$d_1$:Milky\n$d_2$:Sherlock'
#     plt.text(left_lim+0.13,0.95*max_accu,ds_text,bbox=props2,fontsize=11,horizontalalignment='left', verticalalignment='top')
    ds_text = '$d_0$:GreenEyes        $d_1$:Milky      $d_2$:Sherlock'
    plt.text(left_lim+0.12,0.96*max_accu[m],ds_text,bbox=props2,fontsize=11,horizontalalignment='left', verticalalignment='top')

    # legend
    legend_handle = []
    for patch_i in range(2):
        legend_handle.append(mpatches.Patch(color=color_all[patch_i], label=lg_lb[patch_i]))    
    l1 = plt.legend(handles=legend_handle,bbox_to_anchor=(0.,1.02,1.,.102), loc=3,ncol=2,mode="expand", borderaxespad=0.)
    ax = plt.gca().add_artist(l1)
#     plt.legend(line ,['chance'],bbox_to_anchor=(0.,1.005,1.008,.03),loc=1,ncol=1)
    plt.legend(line ,['chance'],bbox_to_anchor=(0.,.83,1.045,0.),loc=9,ncol=1)
     
#     plt.savefig(output_file.format(exp,model,roi)+'.eps', format='eps', dpi=200,bbox_inches='tight')
    plt.savefig(output_file.format(exp,model,roi)+'.pdf', format='pdf', dpi=200,bbox_inches='tight')